Combining Qdrant and LlamaIndex to keep Q&A systems up-to-date


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qdrant/examples/blob/master/llama_index_recency/Qdrant%20and%20LlamaIndex%20%E2%80%94%20A%20new%20way%20to%20keep%20your%20Q%26A%20systems%20up-to-date.ipynb)

#  Introduction

Have you ever been frustrated with an answer engine that is stuck in the past? As our world rapidly evolves, the accuracy of information changes accordingly. Traditional models can become outdated, providing answers that were once accurate but are now obsolete. The cost of outdated knowledge can be high - misinforming users, impacting decision-making, and ultimately undermining trust in your system.

Qdrant and LlamaIndex work together seamlessly, continually adapting your engine to the relentless pace of information change. By mastering these tools, you can transform your applications from static knowledge repositories into dynamic, adaptable knowledge machines. Whether you're a seasoned data scientist or an AI enthusiast, join us on this learning journey - the future of answer engines is here, and it's time to embrace it.

## Learning Outcomes

In this tutorial, you will learn the following:

- 1️⃣ How to build a question-answering system using LlamaIndex and Qdrant.
    - We will load a news dataset, store it with Qdrant client, and load the data into LlamaIndex.
- 2️⃣ How to keep the QA engine updated and improve the ranking system.
    - We will define two postprocessors: Recency and Cohere Rerank; and use these to create various query engines.
- 3️⃣ How to use Node Sources in LlamaIndex to investigate questions and sources on which the answers are based.
    - We will query these engines with various questions and compare their responses.


## Prerequisites

Main Tools
1. `llama_index`: A powerful tool for building large-scale information retrieval systems. [Learn More](https://gpt-index.readthedocs.io/en/latest/getting_started/starter_example.html)
2. `qdrant_client`: A high-performance vector database designed for storing and searching large-scale high-dimensional vectors. In this tutorial, we use Qdrant as our vector storage system.
3. `cohere`: A key reranking service to be used in postprocessing. It takes in a query and a list of texts and returns an ordered array with each text assigned a _new_ relevance score.
4. `OpenAI`: Important for answer generation, as it takes the top few candidates to produce a final answer.
5. `datasets`: Library necessary to import our dataset.
6. `pandas`: Relevant library for data manipulation and analysis.


### Install Packages

Before you start, install the required packages with pip:

In [ ]:
pip install -U datasets

In [ ]:
!pip install llama-index cohere pandas

In [ ]:
!pip install -U qdrant-client

In [4]:
pip install -q cohere llama-index-postprocessor-cohere-rerank

In [ ]:
pip install llama-index-vector-stores-qdrant

Optional: install Rich to make error messages and stack traces easier to read.


In [6]:
# !pip install 'rich[jupyter]'
%load_ext rich

Import your packages

In [7]:
import datetime
import os
import random
from pathlib import Path
from typing import Any

import pandas as pd
from datasets import load_dataset
from IPython.display import Markdown, display_markdown
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext, SimpleDirectoryReader


In [8]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

In [9]:
from llama_index.core.postprocessor import FixedRecencyPostprocessor

In [10]:
from llama_index.vector_stores.qdrant import QdrantVectorStore


In [11]:
Path.ls = lambda x: list(x.iterdir())
random.seed(42)  # This is the answer

### Retrieve API Keys:

Before you start, you must retrieve two API keys for the following services:

1. OpenAI key for LLM. [Link](https://platform.openai.com/account/api-keys)
2. Cohere key for Rerank. [Link](https://dashboard.cohere.ai/api-keys) or additionally, read [Cohere Documentation](https://docs.cohere.com/reference/key).

This tutorial by default uses Qdrant Cloud instead, so you need a third key. You can get it [the Qdrant Cloud main control panel](https://cloud.qdrant.io/)   

If you are running on Colab, you will need to save your API keys under the secrets section of Colab. Adjust accordingly if you are running the notebook in a different environment.




In [12]:
from google.colab import userdata

In [13]:
def check_environment_keys():
    """
    Utility Function that you have the NECESSARY Keys
    """
    if userdata.get("OPENAI_API_KEY") is None:
        raise ValueError(
            "OPENAI_API_KEY cannot be None. Set the key using os.environ['OPENAI_API_KEY']='sk-xxx'"
        )
    if userdata.get("COHERE_API_KEY") is None:
        raise ValueError(
            "COHERE_API_KEY cannot be None. Set the key using os.environ['COHERE_API_KEY']='xxx'"
        )
    if userdata.get("QDRANT_API_KEY") is None:
        print("[Optional] If you want to use the Qdrant Cloud, please get the Qdrant Cloud API Keys and URL")


check_environment_keys()

In [14]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

## Architecture

Our answer engine consists of two main parts:

1. Retrieval - Done with Qdrant
2. Synthesis - Done with OpenAI API

We will use LlamaIndex to make the Query Engine and Qdrant for our Vector Store. Later, we will add components to keep the engine updated and improve ranking after retrieval

The arrow point represents the direction of data flow. The "Query Engine" box encapsulates the postprocessing step to indicate that it's a part of the query engine's function. This diagram is meant to provide a high-level understanding of the process and does not include all the details involved.

![](images/SetupFocus.png)





# Load Sample Dataset

First we need to load our documents. In this example, we will use the [News Category Dataset v3](https://huggingface.co/datasets/heegyu/news-category-dataset). This dataset contains news articles with various fields like `headline`, `category`, `short_description`, `link`, `authors`, and date. Once we load the data, we will reformat it to suit our needs.

In [15]:
dataset = load_dataset("heegyu/news-category-dataset", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/101 [00:00<?, ?B/s]

data.json:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/209527 [00:00<?, ? examples/s]

In [16]:
def get_single_text(k):
    return f"Under the category:\n{k['category']}:\n{k['headline']}\n{k['short_description']}"


df = pd.DataFrame(dataset)
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [17]:
# Assuming `df` is your original dataframe
df["year"] = df["date"].dt.year

category_columns_to_keep = ["POLITICS", "THE WORLDPOST", "WORLD NEWS", "WORLDPOST", "U.S. NEWS"]

# Filter by category
df_filtered = df[df["category"].isin(category_columns_to_keep)]

# Sample data for each year


def sample_func(x):
    return x.sample(min(len(x), 200), random_state=42)


df_sampled = df_filtered.groupby("year").apply(sample_func).reset_index(drop=True)

<ipython-input-17-a898c893970a>:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_filtered.groupby("year").apply(sample_func).reset_index(drop=True)


In [18]:
df_sampled["year"].value_counts()

,count
year,
2014,200
2015,200
2016,200
2017,200
2018,200
2019,200
2020,200
2021,200
2022,200


In [19]:
del df

In [20]:
df = df_sampled

In [21]:
df["text"] = df.apply(get_single_text, axis=1)
df["text"]

,text
0,Under the category:\nWORLDPOST:\nAfghans Don't...
1,Under the category:\nPOLITICS:\nACLU Seeks To ...
2,Under the category:\nPOLITICS:\nWork and Worth...
3,"Under the category:\nPOLITICS:\nJody Hice, Ant..."
4,Under the category:\nPOLITICS:\nCapito Wins We...
...,...
1795,Under the category:\nPOLITICS:\nA Hard-Right R...
1796,Under the category:\nPOLITICS:\nHerschel Walke...
1797,"Under the category:\nU.S. NEWS:\nStocks Fall, ..."
1798,Under the category:\nWORLD NEWS:\nPeru Court O...


In [22]:
df["text"][9]

"Under the category:\nWORLDPOST:\nFreed Taliban Commander Tells Relative He'll Fight Americans Again\n"

In [23]:
df.drop(columns=["year"], inplace=True)

Next, write these documents to text files in a directory. Each document will be written to a text file named after its date.

In [24]:
%%time
write_dir = Path("../data/sample").resolve()
if write_dir.exists():
    [f.unlink() for f in write_dir.ls()]
write_dir.mkdir(exist_ok=True, parents=True)
for index, row in df.iterrows():
    date = str(row["date"]).replace("-", "_")  # replace '-' in date with '_' to avoid issues with file names
    file_path = write_dir / f"date_{date}_row_{index}.txt"
    with file_path.open("w") as f:
        f.write(row["text"])

CPU times: user 273 ms, sys: 107 ms, total: 380 ms
Wall time: 389 ms


In [25]:
# del dataset, df

## Store Dataset with Qdrant Client
We'll be using Qdrant as our vector storage system. Qdrant is a high-performance vector database designed for storing and searching large-scale high-dimensional vectors.

### Local Qdrant Server/Docker + Cloud Instructions
- If you're running a local Qdrant instance with Docker, use `uri`:
  - `uri="http://<host>:<port>"`
  
Here I'll be using the cloud, so I am using the url set to my cloud instance

- Set the API KEY for Qdrant Cloud:
  - `api_key="<qdrant-api-key>"`
  - `url`


In [26]:
from qdrant_client import QdrantClient

client = QdrantClient(
    url="https://5f1a5c6c-7d47-45c3-8d47-d7389b1fad66.eu-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=userdata.get("QDRANT_API_KEY"),
)

print(client.get_collections())

collections=[CollectionDescription(name='sample-movies'), CollectionDescription(name='standard-dense-py'), CollectionDescription(name='NewsCategoryv3PoliticsSample'), CollectionDescription(name='_migration_offsets')]


## Load Data into LlamaIndex
LlamaIndex has a simple way to load documents from a directory. We can define a function to get the metadata from a file name, and pass this function to the `SimpleDirectoryReader` class.

In [27]:
def get_file_metadata(file_name: str):
    """Get file metadata."""
    date_str = Path(file_name).stem.split("_")[1:4]
    return {"date": "-".join(date_str)}


documents = SimpleDirectoryReader(input_files=write_dir.ls(), file_metadata=get_file_metadata).load_data()

In [28]:
len(documents)

1800

Let's look at the date ranges in our dataset:

In [29]:
dates, years = [], []

for document in documents:
    dt = datetime.datetime.fromisoformat(document.extra_info["date"])
    #     print(d)
    try:
        dates.append(dt)
        years.append(dt.year)
    except:
        print(dt)

<ipython-input-29-334b20fa50f4>:4: DeprecationWarning: Call to deprecated function (or staticmethod) extra_info. ('extra_info' is deprecated, use 'metadata' instead.) -- Deprecated since version 0.12.2.
  dt = datetime.datetime.fromisoformat(document.extra_info["date"])


This `date` key is *necessary* for the Recency Postprocessor that we are going to use later.

We have to parse these documents into nodes and create our QdrantVectorStore:

In [30]:
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from llama_index.core import StorageContext


# Define settings globally
Settings.node_parser = SentenceSplitter(chunk_size=512)

vector_store = QdrantVectorStore(client=client, collection_name="NewsCategoryv3PoliticsSample")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Next, we will create our `VectorStoreIndex` from the documents. This operation might take some time as it's creating the index from the documents.

In [31]:
%%time
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

CPU times: user 5.53 s, sys: 705 ms, total: 6.23 s
Wall time: 22.8 s


In [32]:
client.get_collections()


CollectionsResponse(
    collections=[
        CollectionDescription(name='sample-movies'),
        CollectionDescription(name='standard-dense-py'),
        CollectionDescription(name='NewsCategoryv3PoliticsSample'),
        CollectionDescription(name='_migration_offsets')
    ]
)

## Run a Test Query

We have made an index. But as we saw in the diagram, we also need some added functionality to do 3 things:

1. Retrieval
    - Convert the text query into embedding
    - Find the most similar documents
2. Synthesis
    - The LLM (here, OpenAI) texts the question, similar documents and a prompt to give you an answer

In [33]:
query_engine = index.as_query_engine(similarity_top_k=10)

In [34]:
response = query_engine.query("Who is the US President?")
print(response)

The US President is Donald Trump.


In [35]:
response = query_engine.query("Who is the current US President?")
print(response)

The current US President is Donald Trump.



# Adding Postprocessors

LlamaIndex excels at composing Retrieval and Ranking steps.

The intention behind this is to improve answer quality. Let's see if we can use Postprocessors to improve answer quality by using two approaches:
1. Selecting the most recent nodes (Recency).
2. Reranking using a different model (Cohere Rerank).

![](images/RankFocus.png)

Here is what the diagram represents:
1. The user issues a query to the query engine.
2. The query engine, which has been configured with certain postprocessors, performs a search on the vector store based on the query.
3. The query engine then postprocesses the results.
4. The postprocessed results are then returned to the user

### Define a Recency Postprocessor

LlamaIndex allows us to add postprocessors to our query engine. These postprocessors can modify the results of our queries after they are returned from the index. Here, we'll add a recency postprocessor to our query engine. This postprocessor will prioritize recent documents in the results.

We'll define a single type of recency postprocessor: `FixedRecencyPostprocessor`.

In [36]:
recency_postprocessor = FixedRecencyPostprocessor(top_k=1)

### Rerank with Cohere

Cohere Rerank works on the top K results which the Retrieval step from Qdrant returns. While Qdrant works on your entire corpus (here thousands, but Qdrant is designed to work with millions) -- Cohere works with the result from Qdrant. This can improve the search results since it's working on smaller number of entries.

![](images/RerankFocus.png)


Rerank endpoint takes in a query and a list of texts and produces an ordered array with each text assigned a relevance score. We'll define a `CohereRerank` postprocessor and add it to our query engine.

## Defining Query Engines
We'll define four query engines for this tutorial:
1. Just the Vector Store i.e. Qdrant here
1. A recency query engine
1. A reranking query engine
1. And a combined query engine.

The recency query engine uses the `FixedRecencyPostprocessor`, the reranking query engine uses the `CohereRerank` postprocessor, and the combined query engine uses both.

In [37]:
top_k = 10  # set one, reuse from now on, ensures consistency

In [38]:
index_query_engine = index.as_query_engine(
    similarity_top_k=top_k,
)

In [39]:
recency_query_engine = index.as_query_engine(
    similarity_top_k=top_k,
    node_postprocessors=[recency_postprocessor],
)

In [40]:
cohere_rerank = CohereRerank(api_key=os.environ["COHERE_API_KEY"], top_n=top_k)
reranking_query_engine = index.as_query_engine(
    similarity_top_k=top_k,
    node_postprocessors=[cohere_rerank],
)

In [41]:
query_engine = index.as_query_engine(
    similarity_top_k=top_k,
    node_postprocessors=[cohere_rerank, recency_postprocessor],
)

## Querying the Engine
Finally, we can query our engine. Let's ask it "Who is the current US President?" and see the results from each query engine.

In [42]:
# question = "Who is the current US President?"
response = index_query_engine.query("Who is the US President?")
print(response)

The US President is Donald Trump.


The `response` object has a few interesting attributes which help us quickly debug and understand what happened in each of our steps:
1. What source nodes (similar to Document Chunks in Langchain) were used to answer the question
2. What `extra_info` does the index have which we can use? This could also be sent as a payload to Qdrant to filter on (via epoch time) -- but Llama Index does not

Let's unpack that a bit, and we'll use what we learn from `response` to improve our understanding of the query engines and post processors themselves.

Note that `10` which is the top-k parameter we set. This confirms that we retrieved the 10 documents most similar to the question (or more correct: 10 nearest neighbours to the question) and a confidence score.

Can we show this in a more human-readable way?

In [43]:
print(response.get_formatted_sources()[:318])

> Source (Doc id: c6d27f9b-2df9-4127-9d50-4a28f787489b): Under the category:
THE WORLDPOST:
World Leaders React To The Reality Of A Trump Presidency
Many ...

> Source (Doc id: b7ba07bc-ab33-48c7-9d67-ce953328b304): Under the category:
POLITICS:
Six Bullets
We have very little time before the cumulative effect o...




Let's check what is stored in the `extra_info` attribute.

In [60]:
response.metadata


{
    'c6d27f9b-2df9-4127-9d50-4a28f787489b': {'date': '2017-01-28 00:00:00'},
    'b7ba07bc-ab33-48c7-9d67-ce953328b304': {'date': '2014-08-21 00:00:00'},
    '4b06b806-d093-4eef-827f-9a5aa411deb5': {'date': '2016-06-24 00:00:00'},
    '4c199361-5139-4c55-b6b3-d111609c3883': {'date': '2018-12-26 00:00:00'}
}

This has a `date` key-value as a string against the `doc id`

Let's setup some tools to have a question, answer and the responses from the index engine in the same object - this will come handy in a bit for explaining a wrong answer.

In [45]:
def mprint(text: str):
    display_markdown(Markdown(text))


class QAInfo:
    """This class is used to store the question, correct answer and responses from different query engines."""

    def __init__(self, question: str, correct_answer: str, query_engines: dict[str, Any]):
        self.question = question
        self.query_engines = query_engines
        self.correct_answer = correct_answer
        self.responses = {}

    def add_response(self, engine: str, response: str):
        # This method is used to add the response of a query engine to the responses dictionary.
        self.responses[engine] = response

    def compare_responses(self):
        """This function takes in a QAInfo object and a dictionary of query engines, and runs the question through each query engine.
        The responses from each engine are added to the QAInfo object."""
        mprint(f"### Question: {self.question}")

        for engine_name, engine in query_engines.items():
            response = engine.query(self.question)
            self.add_response(engine_name, response)
            mprint(f"**{engine_name.title()}**: {response}")

        mprint(f"Correct Answer is: {self.correct_answer}")

    def node_print(self, index, preview_count=5):
        source_nodes = self.responses[index].source_nodes
        for i in range(preview_count):
            mprint(f"- {source_nodes[i].node.text}")


query_engines = {
    "qdrant": index_query_engine,
    "recency": recency_query_engine,
    "reranking": reranking_query_engine,
    "both": query_engine,
}

In [61]:
question = "Who is the US President?"
correct_answer = "Donald Trump"  # This would normally be determined programmatically.
president_qa_info = QAInfo(question=question, correct_answer=correct_answer, query_engines=query_engines)
president_qa_info.compare_responses()

### Question: Who is the US President?

**Qdrant**: The US President is Donald Trump.

**Recency**: The US President is Donald Trump.

**Reranking**: Obama

**Both**: The US President is Donald Trump.

Correct Answer is: Donald Trump

In [62]:
president_qa_info.node_print(index="recency", preview_count=1)

- Under the category:
POLITICS:
Trump Makes Surprise Iraq Visit After Christmas [UPDATED]
The commander in chief opted for digital holiday greetings, video conferencing with military members around the globe from the Oval Office.

In [63]:
president_qa_info.node_print(index="qdrant", preview_count=1)

- Under the category:
THE WORLDPOST:
World Leaders React To The Reality Of A Trump Presidency
Many of the presidential memorandums and executive decisions will fundamentally affect countries around the globe.

## Impact of how a question is asked

In [64]:
question = "Who is US President in 2022?"
correct_answer = "Joe Biden"  # This would normally be determined programmatically.
current_president_qa_info = QAInfo(
    question=question, correct_answer=correct_answer, query_engines=query_engines
)
current_president_qa_info.compare_responses()

### Question: Who is US President in 2022?

**Qdrant**: Joe Biden

**Recency**: The US President in 2022 is Joe Biden.

**Reranking**: Joe Biden

**Both**: The US President in 2022 is Joe Biden.

Correct Answer is: Joe Biden

### Investigating for Ranking Challenges

We pull the few top documents which from each query engine. To make them easy to read, we've a utility `node_print` here.


💡 We notice that Qdrant (using embeddings) correctly pulls out a few mentions of "2024", "Joe Biden" and "President Joe Biden"

💡 Cohere also re-orders the top 10 candidates to give the top 3 which mention "President Joe Biden".

With Recency, we get an undetermined answer. This is because we're only using the one, most recent result.

## 🎓 Try this now:

> Change the `top_k` value passed to `llama_index` and see how that changes the answers

In [65]:
current_president_qa_info.node_print(index="qdrant", preview_count=3)

- Under the category:
POLITICS:
Joe Biden Says He 'Can't Picture' U.S. Troops Being In Afghanistan In 2022
The president doubled down on his promise to end America's longest-running war at a Thursday press conference, though he said a May 1 deadline seemed unlikely.

- Under the category:
POLITICS:
How A Crowded GOP Field Could Bolster A Trump 2024 Campaign
As Donald Trump considers another White House run, polls show he's the most popular figure in the Republican Party.

- Under the category:
POLITICS:
Biden To Give First State Of The Union Address At Fraught Moment
President Joe Biden aims to navigate the country out a pandemic, reboot his stalled domestic agenda and confront Russia’s aggression.

In [66]:
current_president_qa_info.node_print(index="recency", preview_count=1)

- Under the category:
POLITICS:
How A Crowded GOP Field Could Bolster A Trump 2024 Campaign
As Donald Trump considers another White House run, polls show he's the most popular figure in the Republican Party.

In [67]:
current_president_qa_info.node_print(index="reranking", preview_count=3)

- Under the category:
POLITICS:
Biden To Give First State Of The Union Address At Fraught Moment
President Joe Biden aims to navigate the country out a pandemic, reboot his stalled domestic agenda and confront Russia’s aggression.

- Under the category:
POLITICS:
How A Crowded GOP Field Could Bolster A Trump 2024 Campaign
As Donald Trump considers another White House run, polls show he's the most popular figure in the Republican Party.

- Under the category:
POLITICS:
Joe Biden Says He 'Can't Picture' U.S. Troops Being In Afghanistan In 2022
The president doubled down on his promise to end America's longest-running war at a Thursday press conference, though he said a May 1 deadline seemed unlikely.

## Add a specific Year

That looks interesting. Let's try this question after specifying the year:

In [68]:
question = "Who was the US President in 2010?"
correct_answer = "Barack Obama"  # This would normally be determined programmatically.
president_2010_qa_info = QAInfo(question=question, correct_answer=correct_answer, query_engines=query_engines)
president_2010_qa_info.compare_responses()

### Question: Who was the US President in 2010?

**Qdrant**: Barack Obama was the US President in 2010.

**Recency**: Barack Obama

**Reranking**: Barack Obama was the US President in 2010.

**Both**: Barack Obama

Correct Answer is: Barack Obama

Let's try a different variant of this question, specify a year and see what happens?

In [69]:
question = "Who was the Finance Minister of India under Manmohan Singh Govt?"
correct_answer = "P. Chidambaram"  # This would normally be determined programmatically.
prime_minister_jan2014 = QAInfo(question=question, correct_answer=correct_answer, query_engines=query_engines)
prime_minister_jan2014.compare_responses()

### Question: Who was the Finance Minister of India under Manmohan Singh Govt?

**Qdrant**: The Finance Minister of India under the Manmohan Singh government was P. Chidambaram.

**Recency**: P. Chidambaram

**Reranking**: The Finance Minister of India under the Manmohan Singh government was P. Chidambaram.

**Both**: P. Chidambaram

Correct Answer is: P. Chidambaram

### Observation

In this question: All the engines give the correct answer!

This is despite the fact that the Recency Postprocessor response does not even talk about the Indian Prime Minister! ❌

Qdrant via OpenAI Embeddings and Cohere Rerank do not do that much better

The correct answer comes from OpenAI LLM's knowledge of the world!

In [72]:
prime_minister_jan2014.node_print(index="qdrant", preview_count=3)

- Under the category:
POLITICS:
Robbing Main Street to Prop Up Wall Street:  Why Jerry Brown's Rainy Day Fund Is a Bad Idea
There is no need to sequester funds urgently needed by Main Street to pay for Wall Street's malfeasance. Californians can have their cake and eat it too - with a state-owned bank.

- Under the category:
WORLDPOST:
Cities Need To Get Smarter -- And India's On It

- Under the category:
WORLD NEWS:
Arundhati Roy's New Novel Lays India Bare, Unveiling Worlds Within Our Worlds
Malavika Binny, Jawaharlal Nehru University Wearing two hats at once can be an uncomfortable fit, but it does not seem to

In [73]:
prime_minister_jan2014.node_print(index="recency", preview_count=1)

- Under the category:
WORLD NEWS:
Arundhati Roy's New Novel Lays India Bare, Unveiling Worlds Within Our Worlds
Malavika Binny, Jawaharlal Nehru University Wearing two hats at once can be an uncomfortable fit, but it does not seem to

In [76]:
prime_minister_jan2014.node_print(index="reranking", preview_count=3)

- Under the category:
WORLDPOST:
Cities Need To Get Smarter -- And India's On It

- Under the category:
WORLD NEWS:
Arundhati Roy's New Novel Lays India Bare, Unveiling Worlds Within Our Worlds
Malavika Binny, Jawaharlal Nehru University Wearing two hats at once can be an uncomfortable fit, but it does not seem to

- Under the category:
POLITICS:
The World Bank Must Commit to Food Security
Much will be said about bringing roads, electricity and infrastructure to underdeveloped regions. But how committed is the World Bank to the planet as a whole when it is doling out its loans?

# Recap

- 1️⃣ Crafting a Q&A bot with LlamaIndex and Qdrant
    - We dumped a news dataset, kicked up a Qdrant client, and stuffed our data into a LlamaIndex
- 2️⃣ Keeping our Q&A bot fresh and cranking up the ranking goodness
    - We used a recency postprocessor and a Cohere reranking postprocessor, and put them to work building different query engines
- 3️⃣ Using Node Sources in Llama Index to dig into the Q&A trails
    - We threw a bunch of questions at these engines and saw how they stacked up!

We figured out that recency postprocessing has its perks, but it can leave us hanging when we narrow down the info too much. Plugging in a reranking postprocessor like Cohere can help sort the responses better.